In [34]:
import bs4
import requests
import re
import json
import pprint as pp
from csv import writer
import time
%matplotlib inline
%config IPCompleter.greedy=True
%config Completer.use_jedi = False

In [10]:
url="https://www.bilbasen.dk/brugt/bil?IncludeEngrosCVR=true&PriceFrom=0&includeLeasing=false&Fuel=1"
header={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

In [11]:
retval=requests.get(url, headers=header)

In [12]:
htmlText=retval.text

In [13]:
soup=bs4.BeautifulSoup(htmlText)

In [14]:
elements=soup.findAll("div",class_="row listing listing-plus bb-listing-clickable")

The region

In [15]:
regstuff=elements[1].find("div",class_="listing-region")

The description

In [16]:
descstuff=elements[1].find("div",class_="expandable-box")

The mpg,milage and year

In [17]:
ystuff2=elements[1].findAll("div",class_="listing-data")

The make and model

In [192]:
make=elements[1].findAll("a",class_="darkLink")

The car-id

In [19]:
res2=elements[1].find_all("div", attrs={"data-listingid":True})

NOW TIME FOR PYMYSQL

In [24]:
import pymysql

In [25]:
url="3.xxx.xxx.xxx"
user="xxx"
passwd="xxxxxx"
db="bilbasen"
conn=pymysql.connect(host=url,user=user,passwd=passwd,db=db,autocommit=True)

In [ ]:
print(conn.get_host_info())
cursor=conn.cursor()

In [ ]:
query="select * from carprice2"
cursor.execute(query)

In [29]:
resSet=cursor.fetchall();

In [30]:
retval=(resSet[0])

In [37]:
print(list(retval))

[1, 'Renault', 'Clio IV 0,9 TCe 90 Limited Sport Tourer', 2017, 21.7, 'blue', 113800, datetime.datetime(2021, 11, 21, 18, 24, 57)]


NOW TIME TO CRAWL AND SAVE

### elements=soup.findAll("div",class_="row listing listing-plus bb-listing-clickable")
1. Loop igennem
 1. for hvert item find darkLink
 2. listing-region
 3. listing-price
 4. listing-data (pos1.text er kml er pos2 er km,pos3 er year)
 5. data-listingid ("div", attrs={"data-listingid":True})


In [ ]:
with open('cars3.csv','w', encoding='utf8',newline='') as f:
    thewriter=writer(f)
    header=['car_id','make','cartype','kmpl','kmstand','caryear','color','region','price','datetime']
    thewriter.writerow(header)
    counter=0;
    for el in elements:
        #make og type
        make=el.find("a",class_="darkLink")
        makeList=make.text.split()
        mmake=makeList[0]
        ctype=' '.join(makeList[1:])

        #region
        reg=el.find("div",class_="listing-region")
        rreg=reg.text

        #pris
        price=el.find("div",class_="listing-price")
        nprice=price.text.rstrip("kr.")
        rprice=int(nprice.replace(".",""))

        #kml,km og year
        data=el.findAll("div",class_="listing-data")
        cyear=data[3].text
        km=data[2].text
        rkm=km.replace(".","")
        kml=data[1].text
        tkml=re.sub("[^0-9,]","",kml)
        if len(tkml) > 0:
            rkml=float(tkml.replace(",","."))
        else:
            rkml=0.0

        #description
        desc=el.findAll("div",class_="listing-description")
        rdesc=desc[0].text
        desc="TODO"

        #id
        cid=el.findAll("div", attrs={"data-listingid":True})
        rcid=cid[0]['data-listingid']
        
        dataList=[rcid,mmake,ctype,rkml,rkm,cyear,"none",rreg,rprice,int(time.time())]
        #header=['car_id','make','cartype','kmpl','kmstand','caryear','color','region','price','datetime']
        thewriter.writerow(dataList)

        """
        sqlString="INSERT INTO car (car_id,make,cartype,kmpl,kmstand,caryear,color,cardescription,region) \
        VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        data=(rcid,mmake,ctype,rkml,rkm,cyear,"none",desc,rreg)
        """
        sqlString2="INSERT INTO price (price,timeofchange,car_id) VALUES(%s,now(),%s)"
        data2=(rprice,rcid)
        print(data2)
        counter=counter+1
        try:
            cursor.execute(sqlString2,data2)
            pass
        except conn.Error as e:
            print(e)
            print("Error at "+str(counter)+" Will continue ...")
            continue
